In [3]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from IPython import display
display.set_matplotlib_formats('svg')  # 将图片格式转换成svg格式，清晰

C:\Users\OneRaise\AppData\Local\Temp\ipykernel_20880\2692527289.py:6: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  display.set_matplotlib_formats('svg')  # 将图片格式转换成svg格式，清晰


In [4]:
data = pd.read_feather('data/house_sales.ftr')

In [5]:
data.shape

(164944, 1789)

In [6]:
data.head()

,Id,Address,Sold Price,Sold On,Summary,Type,Year built,Heating,Cooling,Parking,...,Well Disclosure,remodeled,DOH2,SerialX,Full Baths,Tax Legal Lot Number,Tax Legal Block Number,Tax Legal Tract Number,Building Name,Zip
0,2080183300,"11205 Monterey,","$2,000,000",01/31/20,"11205 Monterey, San Martin, CA 95046 is a sing...",SingleFamily,No Data,No Data,No Data,0 spaces,...,None,None,None,None,None,None,None,None,None,95046
1,20926300,"5281 Castle Rd,","$2,100,000",02/25/21,Spectacular Mountain and incredible L.A. City ...,SingleFamily,1951,Central,"Central Air, Dual","Driveway, Driveway - Brick",...,None,None,None,None,None,None,None,None,None,91011
2,19595300,"3581 Butcher Dr,","$1,125,000",11/06/19,Eichler Style home! with Santa Clara High! in ...,SingleFamily,1954,Central Forced Air - Gas,Central AC,"Garage, Garage - Attached, Covered",...,None,None,None,None,None,None,None,None,None,95051
3,300472200,"2021 N Milpitas Blvd,","$36,250,000",10/02/20,"2021 N Milpitas Blvd, Milpitas, CA 95035 is a ...",Apartment,1989,Other,No Data,"Mixed, Covered",...,None,None,None,None,None,None,None,None,None,95035
4,2074492000,"LOT 4 Tool Box Spring Rd,","$140,000",10/19/20,Beautiful level lot dotted with pine trees ro...,VacantLand,No Data,No Data,No Data,0 spaces,...,None,None,None,None,None,None,None,None,None,92561


In [7]:
# 删掉有30%或更多缺失值的数据
null_sum = data.isnull().sum()
data.columns[null_sum < len(data) * 0.3]  # 需要保留的列

Index(['Id', 'Address', 'Sold Price', 'Sold On', 'Summary', 'Type',
       'Year built', 'Heating', 'Cooling', 'Parking', 'Bedrooms', 'Bathrooms',
       'Total interior livable area', 'Total spaces', 'Garage spaces',
       'Home type', 'Region', 'Elementary School', 'Elementary School Score',
       'Elementary School Distance', 'High School', 'High School Score',
       'High School Distance', 'Heating features', 'Parking features',
       'Lot size', 'Parcel number', 'Tax assessed value', 'Annual tax amount',
       'Listed On', 'Listed Price', 'Zip'],
      dtype='object')

In [8]:
# 删除缺失值大于30%的列
data.drop(columns=data.columns[null_sum > len(data) * 0.3], inplace=True)

In [9]:
# 查看每一列的数据类型
data.dtypes

Id                             object
Address                        object
Sold Price                     object
Sold On                        object
Summary                        object
Type                           object
Year built                     object
Heating                        object
Cooling                        object
Parking                        object
Bedrooms                       object
Bathrooms                      object
Total interior livable area    object
Total spaces                   object
Garage spaces                  object
Home type                      object
Region                         object
Elementary School              object
Elementary School Score        object
Elementary School Distance     object
High School                    object
High School Score              object
High School Distance           object
Heating features               object
Parking features               object
Lot size                       object
Parcel numbe

In [10]:
data

,Id,Address,Sold Price,Sold On,Summary,Type,Year built,Heating,Cooling,Parking,...,High School Distance,Heating features,Parking features,Lot size,Parcel number,Tax assessed value,Annual tax amount,Listed On,Listed Price,Zip
0,2080183300,"11205 Monterey,","$2,000,000",01/31/20,"11205 Monterey, San Martin, CA 95046 is a sing...",SingleFamily,No Data,No Data,No Data,0 spaces,...,1.4,None,None,None,None,None,None,None,None,95046
1,20926300,"5281 Castle Rd,","$2,100,000",02/25/21,Spectacular Mountain and incredible L.A. City ...,SingleFamily,1951,Central,"Central Air, Dual","Driveway, Driveway - Brick",...,1.2,Central,"Driveway, Driveway - Brick","13,168 sqft",5870016003,"$1,829,308","$22,330",1/13/2021,"$1,950,000",91011
2,19595300,"3581 Butcher Dr,","$1,125,000",11/06/19,Eichler Style home! with Santa Clara High! in ...,SingleFamily,1954,Central Forced Air - Gas,Central AC,"Garage, Garage - Attached, Covered",...,0.8,"Forced air, Gas","Garage, Garage - Attached, Covered","4,795 sqft",29022100,"$1,125,000","$13,472",9/2/2019,"$1,299,888",95051
3,300472200,"2021 N Milpitas Blvd,","$36,250,000",10/02/20,"2021 N Milpitas Blvd, Milpitas, CA 95035 is a ...",Apartment,1989,Other,No Data,"Mixed, Covered",...,0.9,Other,"Mixed, Covered",4.10 Acres,02201015,"$14,521,205","$175,211",None,None,95035
4,2074492000,"LOT 4 Tool Box Spring Rd,","$140,000",10/19/20,Beautiful level lot dotted with pine trees ro...,VacantLand,No Data,No Data,No Data,0 spaces,...,25.8,None,None,None,None,None,None,None,None,92561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164939,72555199,"88 Lakeshore Ct,","$563,527",02/19/21,STUNNING LAKE VIEW AND BAY VIEW! Exquisite Res...,Condo,1991,"Baseboard, Electric",None,"Carport, Covered, Guest",...,1.3,"Baseboard, Electric","Carport, Covered, Guest",0.05 Acres,5607700878,"$486,031","$8,243",1/15/2021,"$548,000",94804
164940,94643599,"3785 Wilshire Blvd PENTHOUSE 3,","$2,650,000",05/31/18,Arguably the best unit at Solair. This top flo...,Condo,2009,Central,Central,Covered,...,None,Other,Covered,1.23 Acres,5503030207,"$2,757,060","$33,279",2/9/2018,"$2,750,000",90010
164941,300479799,"312 Circuit Way,","$1,357,000",11/04/19,"312 Circuit Way, Mountain View, CA 94043 is a ...",Condo,2019,Other,No Data,"Garage, Garage - Attached, Covered",...,2.1,Other,"Garage, Garage - Attached, Covered","1,651 sqft",16088037,"$1,356,661","$16,427",None,None,94043
164942,15504399,"2 Cape Breton Ct,","$1,400,000",07/23/20,Park Pacifica CHECK OUT THE VIDEO Entering Ca...,SingleFamily,1973,Central Forced Air - Gas,None,"Underground/Basement, Garage - Attached",...,6.2,"Forced air, Gas","Underground/Basement, Garage - Attached","5,401 sqft",022402090,"$272,767","$4,165",5/6/2020,"$1,398,000",94044


In [11]:
# 将价钱相关的列的数据转化成浮点型
currency = ['Sold Price', 'Listed Price', 'Tax assessed value', 'Annual tax amount']
for c in currency:
    data[c] = data[c].replace(r'[$,-]', '', regex=True)
    data[c] = data[c].replace(r'^\s*$', np.nan, regex=True)
    data[c] = data[c].astype(float)

# 将面积大小相关的列转换成浮点型
areas = ['Total interior livable area', 'Lot size']
for c in areas:
    acres = data[c].str.contains('Acres') == True

In [12]:
# 